In [54]:
cd nli/infersent/

[Errno 2] No such file or directory: 'nli/infersent/'
/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli/infersent


In [55]:
import torch
import numpy as np

GLOVE_PATH = '/Volumes/Nitish-Passport/bioasq_files/glove.840B.300d.txt'

In [11]:
model_path = 'saved_models/infersent.allnli.pickle'
model = torch.load(model_path, map_location=lambda storage, loc: storage)

In [12]:
model.set_glove_path(GLOVE_PATH)

In [13]:
sentences = [
    'This is a sample sentence.',
    'The cat likes to walk.',
    'The cat likes to talk.',
    'The dog likes to talk.',
    'This is a completely unrelated sentence.'
]

In [15]:
model.build_vocab(sentences)

Found 17(/17) words with glove vectors
Vocab size : 17


In [18]:
embeddings = model.encode(sentences, bsize=2, tokenize=False, verbose=True)

Nb words kept : 31/36 (86.11 %)
Speed : 7.6 sentences/s (cpu mode, bsize=2)


In [20]:
embeddings.shape

(5, 4096)

In [45]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def sim(sent1, sent2):
    em = model.encode([sent1, sent2], tokenize=False, verbose=True)
    return cosine(em[0], em[1])

In [53]:
model.get_word_dict(sentences).keys()

['a',
 'sentence',
 'This',
 'is',
 'walk',
 'dog',
 '.',
 'sample',
 'to',
 'completely',
 'unrelated',
 'likes',
 'The',
 'cat',
 '</s>',
 'talk',
 '<s>']

## NLI

In [82]:
run data.py

In [140]:
import torch
import numpy as np
from torch.autograd import Variable
from nltk.tokenize import word_tokenize

GLOVE_PATH = '/Volumes/Nitish-Passport/bioasq_files/glove.840B.300d.txt'

In [217]:
s1 = [
    'I do not want to go home.',
    'He likes butter.',
    'He never speaks loud.',
    'The weather is good',
    'What is the matter?',
    'He never speaks loud.',
    'He made two mistakes',
]
s2 = [
    'I wish to go home.',
    'He hates butter.',
    'He never speaks soft.',
    'The weather is bad',
    'What is the matter with you?',
    'He always speaks soft.',
    'He made mistakes twice.',
]

In [218]:
word_dict = get_word_dict(s1 + s2)
word_vec = get_glove(word_dict, GLOVE_PATH)

Found 36(/36) words with glove vectors


In [219]:
nli_net = torch.load('saved_models/nlimodels/model.pickle', map_location='cpu')
nli_net.eval()
outputs = []
for i in range(len(s1)):
    ss1 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s1[i]]]
    ss2 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s2[i]]]
    # ss1 = sorted(ss1, key = lambda x: len(x), reverse=True)
    # ss2 = sorted(ss2, key = lambda x: len(x), reverse=True)
    s1_batch, s1_len = get_batch(ss1, word_vec)
    s2_batch, s2_len = get_batch(ss2, word_vec)
    s1_batch, s2_batch = Variable(s1_batch), Variable(s2_batch)
    output = nli_net((s1_batch, s1_len), (s2_batch, s2_len))
    outputs.append(output.data.max(1)[1].numpy()[0])
outputs

[2, 2, 0, 2, 0, 2, 0]

In [220]:
s1, s2

(['I do not want to go home.',
  'He likes butter.',
  'He never speaks loud.',
  'The weather is good',
  'What is the matter?',
  'He never speaks loud.',
  'He made two mistakes'],
 ['I wish to go home.',
  'He hates butter.',
  'He never speaks soft.',
  'The weather is bad',
  'What is the matter with you?',
  'He always speaks soft.',
  'He made mistakes twice.'])

## Prostrcustus transformation matrix

In [2]:
cd ~/acads/11797/bioasq/nli/infersent/data/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli/infersent/data


In [3]:
import numpy as np

In [4]:
mat = np.load('matmul_glove_med.npy')

In [22]:
mat.shape

(300, 200)

In [23]:
U, s, V = np.linalg.svd(mat, full_matrices=False)

In [24]:
U.shape, s.shape, V.shape

((300, 200), (200,), (200, 200))

In [33]:
tm = U.dot(V)

In [37]:
np.save('prostrcustus_transformation_matrix.npy', tm)

## Embedding Creation

In [1]:
cd ~/acads/11797/bioasq/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq


In [2]:
from dataLoader import DataLoader

input_path = 'input/BioASQ-trainingDataset6b.json'
# input_path = 'input/BioASQ-task6bPhaseB-testset3.json'
data = DataLoader(input_path)

In [3]:
pubmed_vectors_file = '/Volumes/Nitish-Passport/bioasq_files/wikipedia-pubmed-and-PMC-w2v.bin'

input_glove_file = '/Volumes/Nitish-Passport/bioasq_files/glove.840B.300d.txt'
new_glove_file = '/Volumes/Nitish-Passport/bioasq_files/%s_glove.840B.300d.txt' % data.name

In [4]:
cd ~/acads/11797/bioasq/nli/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli


In [5]:
run hypotheses.py

In [6]:
# RerankingParser._parser_model_loaded = False
# rrp = RerankingParser.from_unified_model_dir('/Users/nitish/.local/share/bllipparser/GENIA+PubMed')
# # fetch_and_load('GENIA+PubMed', verbose=True)
# yesno = data.get_questions_of_type('yesno')
# for q in tqdm(yesno):
#     q.assertion_pos = q2s(q.question, rrp)
set_assertions_for_yesno_questions(data)

Model directory: /Users/nitish/.local/share/bllipparser/GENIA+PubMed
Model directory already exists, not reinstalling


100%|██████████| 500/500 [01:06<00:00,  7.53it/s]


In [7]:
yesno = data.get_questions_of_type('yesno')
all_sentences = []
for q in yesno:
    all_sentences.append(q.assertion_pos)
    all_sentences += q.snippet_sentences

In [8]:
cd ~/acads/11797/bioasq/nli/infersent

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli/infersent


In [9]:
run data.py

In [10]:
word_dict = get_word_dict(all_sentences)
tokens = word_dict.keys()

In [11]:
run create_glove_embeddings.py

In [12]:
pubtator_vectors = word2vec.load(pubmed_vectors_file)

In [13]:
import cPickle as pickle

with open('/Users/nitish/acads/11797/bioasq/nli/infersent/data/uncommon_vocab.pkl', 'rb') as fp:
    uncommon_vocab = pickle.load(fp)

In [ ]:
cd data

In [20]:
tm = np.load('prostrcustus_transformation_matrix.npy')

In [21]:
cd ..

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli/infersent


In [23]:
k = 0
with open(new_glove_file, 'w') as fp:
    for word in tokens:
        k += 1
        if k % 5000 == 0:
            print('Finished %d words' % k)
        if word in uncommon_vocab and word in pubtator_vectors:
            vec = tm.dot(pubtator_vectors[word][:, np.newaxis])[:, 0]
            line = ' '.join([word] + [str(i) for i in vec])
            fp.write('%s\n' % line)

Finished 5000 words
Finished 10000 words
Finished 15000 words
Finished 20000 words


In [24]:
# vec = tm.dot(pubtator_vectors['localizes'][:, np.newaxis])[:, 0].shape
# vec.lot_list()

## Run infersent on yesno questions

In [1]:
cd ~/acads/11797/bioasq/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq


In [2]:
run dataLoader.py

In [3]:
# input_path = 'input/BioASQ-trainingDataset5b.json'
# input_path = 'input/phaseB_5b_05.json'
# input_path = 'input/BioASQ-trainingDataset6b.json'
# input_path = 'input/BioASQ-task6bPhaseB-testset3.json'
# input_path = 'input/BioASQ-task6bPhaseB-testset4.json'
input_path = 'input/BioASQ-task6bPhaseB-testset5.json'
data = DataLoader(input_path)

input_glove_file = '/Volumes/Nitish-Passport/bioasq_files/glove.840B.300d.txt'
new_glove_file = '/Volumes/Nitish-Passport/bioasq_files/%s_glove.840B.300d.txt' % data.name
new_glove_file = '/Volumes/Nitish-Passport/bioasq_files/BioASQ-trainingDataset5b_glove.840B.300d.txt'
new_glove_file = '/Volumes/Nitish-Passport/bioasq_files/BioASQ-trainingDataset6b_glove.840B.300d.txt'
yesno_ans_file = 'output/yesno_%s.json' % data.name

In [4]:
cd ~/acads/11797/bioasq/nli/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli


In [5]:
run hypotheses.py

In [6]:
set_assertions_for_yesno_questions(data)

yesno = data.get_questions_of_type('yesno')
all_sentences = []
for q in yesno:
    all_sentences.append(q.assertion_pos)
    all_sentences += q.snippet_sentences

Model directory: /Users/nitish/.local/share/bllipparser/GENIA+PubMed
Model directory already exists, not reinstalling


100%|██████████| 20/20 [00:02<00:00,  7.08it/s]


In [7]:
cd ~/acads/11797/bioasq/nli/infersent

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq/nli/infersent


In [8]:
run data.py

In [9]:
word_dict = get_word_dict(all_sentences)
word_vec = get_glove(word_dict, input_glove_file, new_glove_file)

Found 967(/1035) words with glove vectors


In [10]:
import torch
import numpy as np
from torch.autograd import Variable
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [11]:
len(yesno)

20

In [12]:
nli_net = torch.load('saved_models/nlimodels/model.pickle', map_location='cpu')
nli_net.eval()

for q in yesno:
    outputs = []
    sentences = [i['text'] for i in q.ranked_sentences()[:20]]
    s1, s2 = sentences, [q.assertion_pos] * len(sentences)
#     s2, s1 = sentences, [q.assertion_pos] * len(sentences)
#     if q.exact_answer_ref == 'yes':
#         continue
    total = 0
    for i in range(len(s1)):
        ss1 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s1[i]]]
        ss2 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s2[i]]]
        # ss1 = sorted(ss1, key = lambda x: len(x), reverse=True)
        # ss2 = sorted(ss2, key = lambda x: len(x), reverse=True)
        s1_batch, s1_len = get_batch(ss1, word_vec)
        s2_batch, s2_len = get_batch(ss2, word_vec)
        s1_batch, s2_batch = Variable(s1_batch), Variable(s2_batch)
        output = nli_net((s1_batch, s1_len), (s2_batch, s2_len))
        total += output
        outputs.append(output.data.max(1)[1].numpy()[0])
    print (np.array(outputs) == 0).sum(), (np.array(outputs) == 2).sum(), q.exact_answer_ref #, total.data.max(1)[1].numpy()[0] if total != 0 else 0

/Users/nitish/miniconda2/envs/bioasq/lib/python2.7/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/nitish/miniconda2/envs/bioasq/lib/python2.7/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/nitish/miniconda2/envs/bioasq/lib/python2.7/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve th

0 4 None
2 3 None
0 2 None
2 0 None
0 1 None


Variable containing:
 1.5178 -0.6057 -0.9143
[torch.FloatTensor of size 1x3]

## Eval

In [12]:
nli_net = torch.load('saved_models/nlimodels/model.pickle', map_location='cpu')
nli_net.eval()

def get_answer(n_entail, n_contradict):
    if n_entail > n_contradict:
        return 'yes'
    elif n_contradict > n_entail:
        return 'no'
    else:
        if n_entail == 0:
            return 'no'
        else:
            return 'yes'

entailments = []
answers = []
for q in tqdm(yesno):
    outputs = []
    sentences = [i['text'] for i in q.ranked_sentences()]
    s1, s2 = sentences, [q.assertion_pos] * len(sentences)
    total = 0
    for i in range(len(s1)):
        ss1 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s1[i]]]
        ss2 = [['<s>'] + [word for word in word_tokenize(sent) if word in word_vec] +  ['</s>'] for sent in [s2[i]]]
        s1_batch, s1_len = get_batch(ss1, word_vec)
        s2_batch, s2_len = get_batch(ss2, word_vec)
        s1_batch, s2_batch = Variable(s1_batch), Variable(s2_batch)
        output = nli_net((s1_batch, s1_len), (s2_batch, s2_len))
        total += output
        outputs.append(output.data.max(1)[1].numpy()[0])
    n_entail = (np.array(outputs) == 0).sum()
    n_contradict = (np.array(outputs) == 2).sum()
    total_res = total.data.max(1)[1].numpy()[0] if isinstance(total, Variable) else 0
    q.exact_answer = get_answer(n_entail, n_contradict)
    answers.append((n_entail, n_contradict, q.exact_answer_ref, q.exact_answer))
    print(q.exact_answer, q.exact_answer_ref)
#     entailments.append((n_entail, n_contradict, total_res, q.exact_answer))

# data.eval_yesno()

/Users/nitish/miniconda2/envs/bioasq/lib/python2.7/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/nitish/miniconda2/envs/bioasq/lib/python2.7/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/nitish/miniconda2/envs/bioasq/lib/python2.7/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve th

('yes', None)


 10%|█         | 2/20 [00:12<01:48,  6.05s/it]

('yes', None)


 15%|█▌        | 3/20 [00:22<02:10,  7.66s/it]

...

('yes', None)


 95%|█████████▌| 19/20 [01:52<00:05,  5.93s/it]

('no', None)


100%|██████████| 20/20 [01:57<00:00,  5.86s/it]

('yes', None)


In [44]:
def get_answer(n_entail, n_contradict):
    if n_entail > n_contradict:
        return 'yes'
    elif n_contradict > n_entail:
        return 'no'
    else:
        if n_entail == 0:
            return 'no'
        else:
            return 'yes'

def eval_yesno(questions):
    yes_correct = 0.0
    no_correct = 0.0
    total_yes = 0.0
    total_no = 0.0

    for q in questions:
        if not q.exact_answer_ref or not q.exact_answer:
            continue
        q.exact_answer_ref = q.exact_answer_ref.lower()
        if q.exact_answer_ref == 'yes':
            total_yes += 1
            yes_correct += (q.exact_answer_ref == q.exact_answer)
        if q.exact_answer_ref == 'no':
            total_no += 1
            no_correct += (q.exact_answer_ref == q.exact_answer)

    total_correct = yes_correct + no_correct
    total_q = total_yes + total_no
    total_acc = total_correct / total_q if total_q > 0 else 0
    yes_acc = yes_correct / total_yes if total_yes > 0 else 0
    no_acc = no_correct / total_no if total_no > 0 else 0
    print('Total Accuracy = %2f (%d / %d)' % (total_acc, total_correct, total_q))
    print('YES Accuracy = %2f (%d / %d)' % (yes_acc, yes_correct, total_yes))
    print('No Accuracy = %2f (%d / %d)' % (no_acc, no_correct, total_no))

for itr, q in enumerate(yesno):
    q.exact_answer = get_answer(answers[itr][0], answers[itr][1])

eval_yesno(yesno)

Total Accuracy = 0.568273 (283 / 498)
YES Accuracy = 0.565611 (250 / 442)
No Accuracy = 0.589286 (33 / 56)


In [15]:
cd ~/acads/11797/bioasq/

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq


In [16]:
data.save_yesno_answers(yesno_ans_file)
# data.load_answers_from_file(yesno_ans_file, True)

In [20]:
q = yesno[5]
for q in yesno[:40]:
    print q.question, q.exact_answer, q.exact_answer_ref
#     for i in q.ranked_sentences():
#         print i['text']

Is the protein Papilin secreted? yes yes
Are long non coding RNAs spliced? yes yes
Is RANKL secreted from the cells? yes yes
Does metformin interfere thyroxine absorption? no no
Has Denosumab (Prolia) been approved by FDA? no yes
Is the monoclonal antibody Trastuzumab (Herceptin) of potential use in the treatment of prostate cancer? no yes
Are transcription and splicing connected? yes yes
Is Alu hypomethylation associated with breast cancer? yes yes
Proteomic analyses need prior knowledge of the organism complete genome. Is the complete genome of the bacteria of the genus Arthrobacter available? no yes
Do mutations of AKT1 occur in meningiomas? no yes
Does physical activity influence gut hormones? yes yes
Is irritable bowel syndrome more common in women with endometriosis? yes yes
Does BNP increase after intensive exercise in athletes? yes yes
Are there web based self management strategies for chronic pain ? yes yes
Is Weaver syndrome similar to Sotos? yes yes
Are ultraconserved elemen

In [17]:
cd ../..

/Users/nitish/Documents/Box Sync/cmu/acads/11797/bioasq


In [13]:
yesno_ans_file

'output/yesno_BioASQ-task6bPhaseB-testset5.json'